In [23]:
import pandas as pd
from datetime import datetime, time

# Load the Excel file
file_path = 'Match Dump-DE Take home assignment (1).xlsx'
data = pd.read_excel(file_path)

# Remove the row with column headers (index 111)
data = data[data['Match No.'] != 'Time(IST)(Start Time)']

# Convert 'Time (IST)' to a consistent format
def parse_time(time_str):
    if isinstance(time_str, str):
        try:
            return datetime.strptime(time_str, '%H:%M:%S').time()
        except ValueError:
            try:
                return datetime.strptime(time_str, '%H:%M').time()
            except ValueError:
                return None
    elif isinstance(time_str, time):
        return time_str
    return None

data['Time (IST)'] = data['Time (IST)'].apply(parse_time)

# Handle missing values
data.fillna('Unknown', inplace=True)

# Filter out rows with invalid data
data = data[(data['Match No.'] != 0) & (data['Team A'] != 0) & (data['Team B'] != 0)]

# Update weights
weights = {
    "Series Type": {"World Cup": 20, "Other": 10},
    "Rivalry": {"Ind vs Pak": 25, "Ashes - Eng vs Aus": 20, "Other": 10},
    "Status": {"Live": 15, "Upcoming": 12, "Completed": 8, "Special Match": 10, "Final": 20, "Semi-final": 18, "Quarter-final": 16},
    "Teams": {
        "India": 20, "England": 18, "Australia": 18, "South Africa": 16, "Pakistan": 16,
        "New Zealand": 14, "Sri Lanka": 14, "West Indies": 14, "Afghanistan": 12,
        "St Kitts & Nevis": 10, "Jamaica Tallawahs": 10, "North West": 10,
        "Warriors": 10, "KwaZulu-Natal Inland": 10, "Lions": 10,
        "Others": 8
    },
    "Time": {
        "1700-2030": 15, "1200-1700": 12, "2030-2300": 10, "0900-1200": 8,
        "2300-0100": 6, "0100-0600": 4, "0600-0900": 5
    },
    "Match Category": {"International": 15, "Domestic": 8},
    "Format": {"T20": 15, "One Day": 12, "Test": 10},
    "Is League": {"Yes": 12, "No": 8},
    "Gender": {"Male": 12, "Female": 10}
}

def calculate_trp_score(row, weights):
    score = 0
    
    score += weights["Series Type"].get(row.get("Series Type", "Other"), 10)
    score += weights["Rivalry"].get(row.get("Rivalry", "Other"), 10)
    score += weights["Teams"].get(row.get("Team A", "Others"), weights["Teams"]["Others"])
    score += weights["Teams"].get(row.get("Team B", "Others"), weights["Teams"]["Others"])
    score += weights["Match Category"].get(row.get("Match Category", "Domestic"), 8)
    score += weights["Format"].get(row.get("Match Type", "Other"), 10)
    
    league_event = str(row.get("League/Event", ""))
    score += weights["Is League"].get("Yes" if "League" in league_event else "No", 8)
    
    score += weights["Gender"].get(row.get("Gender", "Male"), 10)
    
    time_slot = row.get("Time (IST)")
    time_slot_weight = 4  # Default to lowest priority
    
    if isinstance(time_slot, time):
        match_time = time_slot.hour * 100 + time_slot.minute
        for slot, weight in weights["Time"].items():
            start, end = map(int, slot.split('-'))
            if start <= match_time <= end:
                time_slot_weight = weight
                break
    
    score += time_slot_weight
    
    # Additional factors
    if row.get("Rivalry") == "Ind vs Pak":
        score += 10  # Extra boost for India vs Pakistan matches
    
    if row.get("Series Type") == "World Cup":
        score += 15  # Extra boost for World Cup matches
    
    return score

# Recalculate the TRP scores
data['TRP Score'] = data.apply(lambda row: calculate_trp_score(row, weights), axis=1)

# Sort the matches by TRP Score in descending order
sorted_matches = data.sort_values(by='TRP Score', ascending=False)

# Display the top 20 matches
print(sorted_matches[['Match No.', 'Date', 'Time (IST)', 'Team A', 'Team B', 'TRP Score', 'Rivalry', 'Series Type']].head(20))

# Print column names to verify available columns
print("\nAvailable columns:")
print(data.columns)

   Match No.                 Date Time (IST)        Team A       Team B  \
85        42  2024-08-21 00:00:00   14:00:00         India      England   
69        20  2024-08-20 00:00:00   14:00:00         India      England   
0          1  2024-08-15 00:00:00   14:00:00         India     Pakistan   
1          8  2024-08-15 00:00:00   14:00:00         India     Pakistan   
32        31  2024-08-17 00:00:00   14:00:00         India     Pakistan   
59        40  2024-08-19 00:00:00   14:00:00         India     Pakistan   
73        48  2024-08-20 00:00:00   14:00:00         India     Pakistan   
16        16  2024-08-16 00:00:00   14:00:00         India     Pakistan   
4         29  2024-08-15 00:00:00   14:00:00       England     Pakistan   
6         43  2024-08-15 00:00:00   14:00:00  South Africa  West Indies   
41         4  2024-08-18 00:00:00   10:00:00  South Africa  West Indies   
38        73  2024-08-17 00:00:00   19:00:00         India    Australia   
10        71  2024-08-15 